In [1]:
import difflib
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import psycopg2
import random
import seaborn as sns
import USC_dataframe_generation as usc

from importlib import reload

In [2]:
conn = psycopg2.connect(host='localhost', dbname='inventory', user='postgres', password='PSGAdmin')
curs = conn.cursor()
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)

In [3]:
heh_cmd = pd.read_sql_query("""SELECT subs_id, sub.class_name, sub.preferred_name, casr_number, ec_number, index_number, 
                                ht."type", hc.classification, hn."name"
                                FROM heh h
                                left join substance sub on sub.id = h.subs_id
                                left join heh_type ht on ht.id = h.heh_type_id
                                left join heh_classification hc on hc.id = h.heh_classif_id
                                left join heh_name hn on hn.id = h.heh_name_id""", conn)

heh_df = pd.DataFrame(heh_cmd, dtype=object)

In [4]:
reg_cmd = pd.read_sql_query("""SELECT reg.id, reg.subs_id, sub.class_name, sub.preferred_name, rco.country, rt."type", rg.general_regulation_name, 
                            rspec.specific_regulation_name, rsub.subspecific_regulation_name, rsc.special_cases_name, addr.additional_information_name, 
                            cid."name", ct."type", regn.names
                            FROM regulations reg
                            LEFT JOIN substance sub ON sub.id = reg.subs_id
                            left join regulation_country rco on rco.id = reg.reg_country_id
                            left join regulation_type rt on rt.id = reg.reg_type_id
                            left join general_regulation rg on rg.id = reg.gen_reg_id
                            left join specific_regulation rspec on rspec.id = reg.spec_reg_id
                            LEFT JOIN subspecific_regulation rsub ON rsub.id = reg.subspec_reg_id
                            left join special_cases_regulation rsc on rsc.id = reg.special_cases_id
                            left join additional_information_regulation addr on addr.id = reg.additional_information_id
                            LEFT JOIN chem_id cid ON cid.chem_type_id = reg.chem_type_id
                            LEFT JOIN chem_type ct ON ct.id = cid.chem_type_id
                            LEFT JOIN regulation_names regn ON regn.id = reg.regulation_id
                            order by reg.id asc""", conn)

reg_df = pd.DataFrame(reg_cmd, dtype=object)

In [5]:
substances = reg_df.subs_id.unique()

In [6]:
cmr_positive = heh_df.loc[(heh_df['type'] == 'CMR') & (~heh_df['name'].isin(['YES','NO','No information','Pending (1)',
                                                                            '(Shall apply from 1 March 2018)',
                                                                            '(31/03/2017)',
                                                                            'Registry of current Harmonised Classification and Labelling intentions by France',
                                                                            'Submitted intention by Netherlands (18/11/2015)',
                                                                            'Submitted intention by Norway (03/10/2011)',
                                                                            'Submitted intention by Germany (22/02/2011)',
                                                                            'Submitted intention by Norway (24/06/2014)',
                                                                            'Amendment 2016/1179', 'Pending','particle diameter < 1 mm)',
                                                                            'Submitted intention by Italy (22/08/2012)',
                                                                            'Submitted intention by Sweden (14/11/2014)'])), 'name'].unique()
pbt_ann = heh_df.loc[(heh_df['type'] == 'PBT') & (~heh_df['name'].isin(['YES','NO','No information','Pending (2)'])), 'name'].unique()
vpvb_ann = heh_df.loc[(heh_df['type'] == 'vPvB') & (~heh_df['name'].isin(['NO','Pending (2)','YES','No information'])), 'name'].unique()
endoc_ann = heh_df.loc[(heh_df['type'] == 'Endocrine_disruptor') & (~heh_df['name'].isin(['YES','NO','Pending (3)','No information'])), 'name'].unique()
sens_ann = heh_df.loc[(heh_df['type'] == 'Sensitiser') & (~heh_df['name'].isin(['YES','NO','No information','(Shall apply from 1 March 2018)',
                                                                                'Pending (3)','Pending(3)','Submitted intention by Slovenia (09/01/2015)',
                                                                               'Submitted intention by United Kingdom (01/02/2017)',
                                                                               'Amendment 2016/1179','(05/01/2016)','particle diameter < 1 mm)',
                                                                               'Proposed future entry in Annex VI of CLP Regulation',
                                                                               'Registry of current Harmonised Classification and Labelling intentions by Norway',
                                                                               'Removal of','SCL ≥ 0.06 %'])), 'name'].unique()
other_ann = heh_df.loc[(heh_df['type'] == 'Other') & (~heh_df['name'].isin(['No information'])), 'name'].unique()

In [7]:
pbt_vpvb_pos = reg_cmd[(reg_cmd['general_regulation_name'].isin(['pbt_vpvb'])) &
                      (~reg_cmd['names'].isin(['No information', 'Not PBT', 'Not vPvB', 'No information available',
                                              '(See REACH part-Candidate List section)','Conclusion: not fulfilling PBT & vPvB criteria',
                                   'Conclusion: deferred the decision on the substance from the list of potential PBT or vPvB substances.',
                                              'Conclusion: not fulfilling PBT  criteria','Conclusion: not fulfilling POP  criteria']))].names.unique()

In [8]:
endoc_pos = reg_cmd[(reg_cmd['general_regulation_name'].isin(['endocrine_disruptors']))&
                   (~reg_cmd['names'].isin(['Not included','No information available']))].names.unique()

In [9]:
new_cmr = ['Toxic for reproduction (Article 57c)','Toxic for reproduction (Article 57c):','Toxic for reproduction','Scope: ED, CMR','Scope: CMR',
           'Carcinogenic (Article 57a)','Carcinogenic (Category 1B)','Carcinogenic  1B','Toxic for reproduction (category 1B)',
           'Mutagenic (Article 57b)','Carcinogenic (category 1A)','Mutagenic (category 1B)',]
new_pbt_vpvb = ['PBT (Article 57 d)','Scope: vPvB (Article 57 e)', 'vPvB (Article 57 e)','Scope: PBT']
other_new = ['Equivalent level of concern having probable serious effects to environment (Article 57 f):','Scope: EQC',
             'Equivalent level of concern having probable serious effects to environment (Article 57 f)']
new_endoc = ['Scope: ED, CMR']

cmr_positive = np.concatenate((cmr_positive, new_cmr), axis=0)
pbt_vpvb_pos = np.concatenate((pbt_vpvb_pos, new_pbt_vpvb), axis=0)
endoc_pos = np.concatenate((endoc_pos,new_endoc), axis=0)
other_ann = np.concatenate((other_ann, other_new), axis=0)

In [10]:
vpvb_pos = ['vPvB substance', 'PBT/ vPvB Substance',
       'PBT/ vPvB substance', 'Rapporteur: Austria', 'Rapporteur: Norway',
       'Rapporteur: United Kingdom', 'Rapporteur: Denmark',
       'Rapporteur: Spain', 'Rapporteur: Germany',
       'Rapporteur: France',
       'Conclusion: Under evaluation', 'Rapporteur: Netherlands',
       'Rapporteur: Sweden',
       'Conclusion: fulfilling PBT & vPvB criteria',
       'Conclusion: fulfilling PBT & vPvB criteria & POP']

In [11]:
pbt_pos = ['PBT Substance', 'PBT/ vPvB Substance',
       'PBT/ vPvB substance', 'Rapporteur: Austria', 'Rapporteur: Norway',
       'Rapporteur: United Kingdom', 'Rapporteur: Denmark',
       'Rapporteur: Spain', 'Rapporteur: Germany',
       'Conclusion: Fulfilling PBT criteria & POP','Rapporteur: France',
       'Conclusion: Under evaluation', 'Rapporteur: Netherlands',
       'Conclusion: fulfilling PBT criteria', 'Rapporteur: Sweden',
       'Conclusion: fulfilling PBT & vPvB criteria',
       'Conclusion: fulfilling PBT & vPvB criteria & POP']

In [12]:
%%time
cmr_dict = usc.fixed_generate_dataframe(substance_id_list=substances,endpoint='CMR', regulations_df=reg_df, endpoint_annotations=cmr_positive)
pbt_dict = usc.fixed_generate_dataframe(substance_id_list=substances,endpoint='PBT', regulations_df=reg_df, endpoint_annotations=pbt_pos)
vpvb_dict = usc.fixed_generate_dataframe(substance_id_list=substances,endpoint='vPvB', regulations_df=reg_df, endpoint_annotations=vpvb_pos)
endoc_dict = usc.fixed_generate_dataframe(substance_id_list=substances,endpoint='Endocrine_disruptor', regulations_df=reg_df, endpoint_annotations=endoc_pos)
sens_dict = usc.fixed_generate_dataframe(substance_id_list=substances,endpoint='Sensitiser', regulations_df=reg_df, endpoint_annotations=sens_ann)
other_dict = usc.fixed_generate_dataframe(substance_id_list=substances,endpoint='Other', regulations_df=reg_df, endpoint_annotations=other_ann)

CPU times: user 1h 18min 18s, sys: 705 ms, total: 1h 18min 19s
Wall time: 1h 18min 18s


In [13]:
cmr_df = pd.DataFrame(data=cmr_dict)
pbt_df = pd.DataFrame(data=pbt_dict)
vpvb_df = pd.DataFrame(data=vpvb_dict)
endoc_df = pd.DataFrame(data=endoc_dict)
sens_df = pd.DataFrame(data=sens_dict)
other_df = pd.DataFrame(data=other_dict)

In [14]:
to_concat_df = [cmr_df, pbt_df, vpvb_df,endoc_df,sens_df]
concat_df = pd.concat(to_concat_df)

In [15]:
for subs_id in substances:
    if other_df[(other_df['subs_id'] == subs_id) & (other_df['name'].isin(['NO','No information']))].empty:
        other_df.loc[other_df['subs_id'] == subs_id,:] = other_df.loc[(other_df['subs_id'] == subs_id) & 
                                                                      (~other_df['name'].isin(concat_df[concat_df['subs_id'] == subs_id].name.unique())),:]

In [16]:
new_other_df = other_df.dropna().reset_index().drop(columns=['index'])

In [17]:
final_to_concat =[concat_df, new_other_df]
final_df = pd.concat(final_to_concat)
sorted_concat_df = final_df.sort_values(by=['subs_id', 'endpoint_type']).reset_index().drop(columns=['index'])

In [18]:
sorted_concat_df.to_pickle('final_generated_fixed_heh_df.pkl')

In [19]:
sorted_concat_df = pd.read_pickle('final_generated_fixed_heh_df.pkl')

In [20]:
endpoint_list = ['CMR','PBT','vPvB','Endocrine_disruptor','Sensitiser','Other']

In [21]:
ann_info_list = ['YES','NO','No information','Pending', 'Pending (1)','Pending (2)',
                    'Pending (3)', 'Pending (3a)','Pending (3b)','Pending (3c)','Pending (4)']

In [22]:
for ep in endpoint_list:
    ep_dict = {'Anotaciones':[],'Inventario':[],'Generadas':[]}
    for ann in ann_info_list:
        total_original = heh_df.loc[(heh_df['type'] == ep) & (heh_df['name'] == ann),'name'].count()
        total_generadas = sorted_concat_df.loc[(sorted_concat_df['endpoint_type'] == ep) & (sorted_concat_df['name'] == ann),'name'].count()
        ep_dict['Anotaciones'].append(ann)
        ep_dict['Inventario'].append(total_original)
        ep_dict['Generadas'].append(total_generadas)
    
    ep_df = pd.DataFrame(data=ep_dict,dtype=object)
    ep_df.set_index('Anotaciones')
#     if os.path.isfile('Comparación_anotaciones_final_arreglado.xlsx'):
#         with pd.ExcelWriter('Comparación_anotaciones_final_arreglado.xlsx', mode='a') as writer:
#              ep_df.to_excel(writer, sheet_name=ep)
#     else:
#         with pd.ExcelWriter('Comparación_anotaciones_final_arreglado.xlsx', mode='w') as writer:
#              ep_df.to_excel(writer, sheet_name=ep)